## Исследование поведения пользователей мобильной игры Quiz Freeze

**Цель**<br>
Исследовать поведение пользователей в обновлённом приложении (игра Quiz Freeze).

**Постановка задачи**<br>
Проверка следующих гипотез:
- наличие зависимости между выбранным уровнем сложности (selected_level) и вероятностью оплаты;
- наличие/отсутствие разницы во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности (selected_level).

**Под исследование подпадает следующая выборка данных:**<br>
Пользователи, зарегестрированные в 2018 году (с 1 января по 31 декабря 2018 года включительно).

**Исходные данные**<br>
Два файла - таблицы в формате csv:
- таблица Event: хранит данные о событиях, которые совершают пользователи;
- таблица purchase: хранит данные об оплатах, которые совершают пользователи.



**Выгрузка и предобработка данных**

In [1]:
# Импортируем библиотеку Pandas, выгружаем таблицы для дальнейшего исследования и создаем их копии

import pandas as pd
events = pd.read_csv('C:/mine/IDE/GD3/7_4_Events.csv', sep=',')
events_df = events.copy()

purchase = pd.read_csv('C:/mine/IDE/GD3/purchase.csv', sep=',')
purchase_df = purchase.copy()


In [2]:
# Выводим первые строки таблицы events для визуального ознакомления с данными
events_df.head()

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585


In [5]:
# делаем выборку пользователей, зарегестрированых в 2018 году

year_2018 = (events_df['start_time'] >= '2018-01-01') & (events_df['start_time'] < '2019-01-01') & (events_df['event_type'] == 'registration')
users_2018 = events_df[year_2018]['user_id'].to_list()
events_df= events_df[events_df['user_id'].isin(users_2018)]

In [6]:
# Выводим детальную информацию по таблице events
events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              66959 non-null  int64  
 1   event_type      66959 non-null  object 
 2   selected_level  8342 non-null   object 
 3   start_time      66959 non-null  object 
 4   tutorial_id     32954 non-null  float64
 5   user_id         66959 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 3.6+ MB


In [8]:
# Из представленной информации видно, что столбец start_time имеет тип данных object, 
# который необходимо изменить на datetime для дальнейшей работы с таблицей

events_df['start_time'] = pd.to_datetime(events_df['start_time'], dayfirst= True)

In [9]:
# Выводим первые строки таблицы purchase для визуального ознакомления с данными
purchase_df.head()

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100


In [12]:
# Из представленной информации видно, что столбец event_datetime необходимо изменить на datetime для дальнейшей работы с таблицей

purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'], dayfirst= True)

In [13]:
# делаем выборку пользователей, зарегестрированых в 2018 году
purchase_df= purchase_df[purchase_df['user_id'].isin(users_2018)]

In [14]:
# выводим основные статистические свойства столбцов таблицы purchase
purchase_df.describe()

,id,user_id,amount
count,1600.000000,1600.000000,1600.000000
mean,17645.505625,37752.766250,110.734375
std,462.038637,5822.621784,54.696628
min,16845.000000,27845.000000,25.000000
25%,17245.750000,32815.750000,50.000000
50%,17645.500000,37633.500000,100.000000
75%,18045.250000,43023.000000,150.000000
max,18452.000000,47742.000000,300.000000


## Проверка первой гипотезы: 
### наличие зависимости между выбранным уровнем сложности (selected_level) и вероятностью оплаты

**Выделение групп пользователей с разным уровнем сложности (selected_level)**

Для дальнейшей работы производим объединение таблиц events и purchase в единую таблицу total_events_df

In [15]:
# Перед объединением в таблицу purchase_df добавляем столбец event_type с обозначением purchase
purchase_df['event_type'] = 'purchase'

# перед объединением переименновываем столбцы с одинаковыми наименованиями
events_df = events_df.rename(columns={"id": "event_id"})
purchase_df = purchase_df.rename(columns={"id": "purchase_id", "event_datetime": "start_time"})

# обьединяем таблицы в общую: total_events_df
total_events_df = pd.concat([events_df,purchase_df],sort=False)
total_events_df['start_time'] = pd.to_datetime(total_events_df['start_time'])

# в итоговой таблице производим сброс индексов и сортировку событий по возрастанию времени
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

# выводим первые строки итоговой таблицы total_events_df
total_events_df.head()

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,amount
0,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaN
1,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaN
2,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaN
3,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaN
4,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaN


In [16]:
# определяем уровни сложности
total_events_df['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

Таким образом получено три группы пользователей, исходя из уровня сложности (selected_level):<br>
 - easy
 - medium
 - hard

In [18]:
# Создаем выборку по группе пользователей easy
mask_easy = total_events_df['selected_level'] == 'easy'
level_easy = total_events_df[mask_easy]['user_id'].to_list()
level_easy_count = len(level_easy)

# Создаем выборку по группе пользователей medium
mask_medium= total_events_df['selected_level'] == 'medium'
level_medium = total_events_df[mask_medium]['user_id'].to_list()
level_medium_count = len(level_medium)

# Создаем выборку по группе пользователей hard
mask_hard= total_events_df['selected_level'] == 'hard'
level_hard = total_events_df[mask_hard]['user_id'].to_list()
level_hard_count = len(level_hard)

# Делаем фильтрацию всех полученных выборок
level_easy_df  = total_events_df[total_events_df['user_id'].isin(level_easy)]
level_medium_df  = total_events_df[total_events_df['user_id'].isin(level_medium)]
level_hard_df  = total_events_df[total_events_df['user_id'].isin(level_hard)]

**Расчет показателей по трем группам пользователей**

In [19]:
# Расчет количества оплат пользователей уровня easy
mask_easy_purchase = level_easy_df['event_type'] == 'purchase'
easy_level_purchase = level_easy_df[mask_easy_purchase]['event_type'].count()

# Расчет количества оплат пользователей уровня medium
mask_medium_purchase = level_medium_df['event_type'] == 'purchase'
medium_level_purchase = level_medium_df[mask_medium_purchase]['event_type'].count()

# Расчет количества оплат пользователей уровня hard
mask_hard_purchase = level_hard_df['event_type'] == 'purchase'
hard_level_purchase = level_hard_df[mask_hard_purchase]['event_type'].count()

In [20]:
# Расчет процента оплат пользователей уровня easy
percent_purchase_easy_level = easy_level_purchase / level_easy_count

# Расчет процента оплат пользователей уровня medium
percent_purchase_medium_level = medium_level_purchase / level_medium_count

# Расчет процента оплат пользователей уровня hard
percent_purchase_hard_level = hard_level_purchase / level_hard_count

In [21]:
# Расчет суммы оплат пользователей уровня easy
level_easy_purchase_sum = level_easy_df[mask_easy_purchase]['amount'].sum()

# Расчет суммы оплат пользователей уровня medium
level_medium_purchase_sum = level_medium_df[mask_medium_purchase]['amount'].sum()

# Расчет суммы оплат пользователей уровня hard
level_hard_purchase_sum = level_hard_df[mask_hard_purchase]['amount'].sum()

In [22]:
# Расчет среднего чека пользователей уровня easy
check_easy_level = level_easy_purchase_sum / easy_level_purchase

# Расчет среднего чека пользователей уровня medium
check_medium_level = level_medium_purchase_sum / medium_level_purchase

# Расчет среднего чека пользователей уровня hard
check_hard_level = level_hard_purchase_sum / hard_level_purchase

**Данныепо трем группам пользователей**

In [25]:
print('Количество пользователей')
print( 'Уровень easy:',level_easy_count)
print( 'Уровень medium:',level_medium_count)
print( 'Уровень hard:',level_hard_count)
print('_______')

print('Количество оплат от пользователей')
print( 'Уровень easy:',easy_level_purchase)
print( 'Уровень medium:',medium_level_purchase)
print( 'Уровень hard:',hard_level_purchase)
print('_______')

print('Процент оплат от пользователей')
print(
    "Уровень easy: {:.2%}".format(
        percent_purchase_easy_level
    )
)
print(
    "Уровень medium: {:.2%}".format(
        percent_purchase_medium_level
    )
)
print(
    "Уровень hard: {:.2%}".format(
        percent_purchase_hard_level
    )
)
print('_______')

print('Общая сумма оплат от пользователей')
print('Уровень easy:', round(
    level_easy_purchase_sum
    )
)
print('Уровень medium:', round(
    level_medium_purchase_sum
    )
)
print('Уровень hard:', round(
    level_hard_purchase_sum
    )
)
print('_______')

print('Средний чек пользователей')
print('Уровень easy:', round(
    check_easy_level
    )
)
print('Уровень medium:', round(
    check_medium_level
    )
)
print('Уровень hard:', round(
    check_hard_level
    )
)

Количество пользователей
Уровень easy: 2448
Уровень medium: 4645
Уровень hard: 1249
_______
Количество оплат от пользователей
Уровень easy: 189
Уровень medium: 969
Уровень hard: 442
_______
Процент оплат от пользователей
Уровень easy: 7.72%
Уровень medium: 20.86%
Уровень hard: 35.39%
_______
Общая сумма оплат от пользователей
Уровень easy: 21725
Уровень medium: 106125
Уровень hard: 49325
_______
Средний чек пользователей
Уровень easy: 115
Уровень medium: 110
Уровень hard: 112


## Проверка второй гипотезы: 
#### наличие/отсутствие разницы во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности (selected_level)

In [26]:
# создаем таблицу level_choice_df, которая содержит выбор уровня сложности
level_choice_df = total_events_df[total_events_df['event_type'] == 'level_choice']
level_choice_df = total_events_df[total_events_df["selected_level"].isin(['hard','medium','easy'])]

# проверяем среднее число выбора уровня сложности на одного пользователя
level_choice_df['user_id'].value_counts().mean()

1.0

In [27]:
# убираем ненужные столбцы и переименовываем столбец start_time в level_choice_time
level_choice_df = level_choice_df[["user_id","selected_level","start_time"]].rename(
    columns={"start_time": "level_choice_time"}
)
level_choice_df.head()

,user_id,selected_level,level_choice_time
19,27835,medium,2018-01-01 20:37:22
23,27839,hard,2018-01-01 22:37:50
36,27840,medium,2018-01-02 05:18:42
38,27845,hard,2018-01-02 06:19:18
45,27842,easy,2018-01-02 08:46:03


In [28]:
# создаем таблицу purch_df, которая содержит только сведения об оплатах (event_type = purchase).
purch_df= total_events_df[total_events_df['event_type'] == 'purchase']
purch_df['user_id'].value_counts().mean()

# убираем ненужные столбцы и переименовываем столбец start_time в registration_time 
purch_df = purch_df[["user_id","start_time"]].rename(
    columns={"start_time": "purchase_time"}
)

purch_df.head()

,user_id,purchase_time
66959,27845,2018-01-03 18:53:43
66960,27865,2018-01-04 14:46:10
66961,27911,2018-01-07 08:19:12
66962,27910,2018-01-07 12:11:34
66963,27940,2018-01-07 13:16:41


In [29]:
# Объединяем таблицы level_choice_df и purch_df
merged_df = level_choice_df.merge(
    purch_df, on="user_id", how="inner"
)

merged_df.head()

,user_id,selected_level,level_choice_time,purchase_time
0,27845,hard,2018-01-02 06:19:18,2018-01-03 18:53:43
1,27865,hard,2018-01-04 05:56:32,2018-01-04 14:46:10
2,27884,easy,2018-01-04 16:18:39,2018-01-08 19:37:34
3,27910,hard,2018-01-05 11:59:50,2018-01-07 12:11:34
4,27911,hard,2018-01-05 17:39:02,2018-01-07 08:19:12


In [30]:
# Расчет разницы во времени между событием оплаты (purchase_time) и событием выбора уровня сложности (registration_time).print
# Результаты расчета выделяем в отдельный столбец timedelta
merged_df["timedelta"] = (
    merged_df["purchase_time"] - merged_df["level_choice_time"])

merged_df.head()

,user_id,selected_level,level_choice_time,purchase_time,timedelta
0,27845,hard,2018-01-02 06:19:18,2018-01-03 18:53:43,1 days 12:34:25
1,27865,hard,2018-01-04 05:56:32,2018-01-04 14:46:10,0 days 08:49:38
2,27884,easy,2018-01-04 16:18:39,2018-01-08 19:37:34,4 days 03:18:55
3,27910,hard,2018-01-05 11:59:50,2018-01-07 12:11:34,2 days 00:11:44
4,27911,hard,2018-01-05 17:39:02,2018-01-07 08:19:12,1 days 14:40:10


In [31]:
# определяем среднее время между событием оплаты и событием выбора уровня сложности
print('Среднее время между событием оплаты и событием выбора уровня сложности') 
print('Для всех уровней сложности =', merged_df["timedelta"].mean())
print('Для уровня сложности hard =', merged_df[merged_df["selected_level"] == "hard"]["timedelta"].mean())
print('Для уровня сложности medium =', merged_df[merged_df["selected_level"] == "medium"]["timedelta"].mean())
print('Для уровня сложности easy =', merged_df[merged_df["selected_level"] == "easy"]["timedelta"].mean())


Среднее время между событием оплаты и событием выбора уровня сложности
Для всех уровней сложности = 3 days 17:52:17.719375
Для уровня сложности hard = 3 days 07:20:41.420814479
Для уровня сложности medium = 3 days 23:14:13.165118679
Для уровня сложности easy = 3 days 14:58:52.941798941


In [32]:
# создаем таблицу registration_df, в который содержится только информация о регистрациях (event_type = registration)
registration_df = total_events_df[total_events_df['event_type'] == 'registration']

# проверяем среднее число событий типа registration на одного пользователя.
registration_df['user_id'].value_counts().mean()

1.0

In [33]:
# убираем ненужные столбцы и переименовываем столбец start_time в столбец registration_time
registration_df = total_events_df[total_events_df['event_type'] == 'registration']
registration_df['user_id'].value_counts().mean()
registration_df = registration_df[["user_id", "start_time"]].rename(
    columns={"start_time": "registration_time"}
)
registration_df.head()

,user_id,registration_time
0,27832,2018-01-01 03:48:40
1,27833,2018-01-01 04:07:25
2,27834,2018-01-01 08:35:10
3,27835,2018-01-01 11:54:47
4,27836,2018-01-01 13:28:07


Просматриваем данные ранее созданных таблиц по уровням сложности

In [34]:
level_easy_df.head()

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,amount
25,80333.0,registration,NaN,2018-01-01 23:18:46,NaN,27842,NaN,NaN
32,80340.0,tutorial_start,NaN,2018-01-02 04:55:11,31513.0,27842,NaN,NaN
33,80341.0,tutorial_finish,NaN,2018-01-02 04:57:01,31513.0,27842,NaN,NaN
40,80348.0,registration,NaN,2018-01-02 07:03:55,NaN,27849,NaN,NaN
45,80353.0,level_choice,easy,2018-01-02 08:46:03,NaN,27842,NaN,NaN


In [35]:
level_medium_df.head()

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,amount
3,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaN
8,80316.0,tutorial_start,NaN,2018-01-01 15:00:51,31506.0,27835,NaN,NaN
9,80317.0,tutorial_finish,NaN,2018-01-01 15:06:15,31506.0,27835,NaN,NaN
19,80327.0,level_choice,medium,2018-01-01 20:37:22,NaN,27835,NaN,NaN
20,80328.0,pack_choice,NaN,2018-01-01 20:38:43,NaN,27835,NaN,NaN


In [36]:
level_hard_df.head()

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,amount
14,80322.0,registration,NaN,2018-01-01 18:24:01,NaN,27839,NaN,NaN
15,80323.0,tutorial_start,NaN,2018-01-01 19:11:36,31509.0,27839,NaN,NaN
16,80324.0,tutorial_finish,NaN,2018-01-01 19:16:32,31509.0,27839,NaN,NaN
23,80331.0,level_choice,hard,2018-01-01 22:37:50,NaN,27839,NaN,NaN
24,80332.0,pack_choice,NaN,2018-01-01 22:42:54,NaN,27839,NaN,NaN


In [38]:
# В каждой таблице делаем выборку по событиям с event_type = purchase (оплата).
mask1_1 = level_easy_df['event_type'] == 'purchase'
level_easy_df = level_easy_df[mask1_1]

mask1_2 = level_medium_df['event_type'] == 'purchase'
level_medium_df = level_medium_df[mask1_2]

mask1_3 = level_hard_df['event_type'] == 'purchase'
level_hard_df = level_hard_df[mask1_3]

# Делаем по каждой таблице проверку количества событий на одного пользователя
print(level_easy_df['user_id'].value_counts().mean())
print(level_medium_df['user_id'].value_counts().mean())
print(level_hard_df['user_id'].value_counts().mean())

1.0
1.0
1.0


In [39]:
# из каждой таблицы убираем ненужные столбцы
level_easy_df = level_easy_df[["user_id", "start_time"]].rename(
    columns={"start_time": "purchase_time"}
)

level_medium_df = level_medium_df[["user_id", "start_time"]].rename(
    columns={"start_time": "purchase_time_medium"}
)

level_hard_df = level_hard_df[["user_id", "start_time"]].rename(
    columns={"start_time": "purchase_time_hard"}
)

In [40]:
# объеденяем каждую из трех таблиц по уровням с таблицей registration_df 
merged_easy_df = registration_df.merge(
    level_easy_df, on="user_id", how="inner"
)

merged_medium_df= registration_df.merge(
    level_medium_df, on="user_id", how="inner"
)

merged_hard_df= registration_df.merge(
    level_hard_df, on="user_id", how="inner"
)

In [41]:
# в каждой полученной таблице делаем расчет разницы во времени между событием оплаты (purchase_time) и событием регистрации (registration_time)
# полученные данные сохраняем в столбце timedelta
merged_easy_df["timedelta"] = (
    merged_df["purchase_time"] - merged_easy_df["registration_time"]
)

merged_medium_df["timedelta"] = (
    merged_medium_df["purchase_time_medium"] - merged_medium_df["registration_time"]
)

merged_hard_df["timedelta"] = (
    merged_hard_df["purchase_time_hard"] - merged_hard_df["registration_time"]
)

In [43]:
# расчитываем среднее время между событием регистрации пользователя и событием оплаты на всех уровнях слоджности:
print('Среднее время между событием регистрации пользователя и событием оплаты на всех уровнях сложности')
print('Уровень easy:', merged_easy_df["timedelta"].mean())
print('Уровень medium:', merged_medium_df["timedelta"].mean())
print('Уровень hard:', merged_hard_df["timedelta"].mean())

Среднее время между событием регистрации пользователя и событием оплаты на всех уровнях сложности
Уровень easy: 3 days 22:10:23.211640211
Уровень medium: 4 days 06:12:06.576883384
Уровень hard: 3 days 14:55:19.257918552


## Выводы

**1. Проверка первой гипотезы:  наличие зависимости между выбранным уровнем сложности (selected_level) и вероятностью оплаты.**<dr>
- максимальное количество пользователей содержится в категории уровня medium ( в 2,2 раза превышает количество пользователей уровня hard и в 5 раз превышает количество пользователей уровня easy)
- максимальный средний чек у категорий различается незначительно
- максимальная сумма оплат поступила также от категории пользователей medium, что закономерно, т.к. это самая многочисленная категория
- максимальный процент оплат от пользователей показала группа уровня hard. При этом процент уровня hard в 4 раза выше процента уровня easy

**Существует прямая зависимость между уровнем сложности и вероятностью оплаты. Чем выше уровень сложности тем больше вероятность оплаты.**


**2. Проверка второй гипотезы: наличие/отсутствие разницы во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности (selected_level).**<dr>
- максимальная разница во времени между событиями наблюдается у группы пользователей medium (более 4 дней и 6 часов)
- минимальная разница наблюдается у группы пользователей hard (3 дня и почти 15 часов)
- разница группы пользователей easy оставляет 3 дня 22 часа, что на 8 часов меньше, чем у группы пользователей medium и на 7 часов больше, чем у группы пользователей hard
- максимальное различие: между группой пользователей medium и группой пользователей hard (около 16 часов)

**Разница во времени между событием регистрации и событием оплаты для разных групп пользователей существует (от 7 до 16 часов)**